In [1]:
import pandas as pd
import numpy as np
import pickle as pkl

from sklearn import linear_model
from sklearn.metrics  import mean_squared_error as mse

from bokeh.plotting import figure, show, output_notebook
from bokeh.charts import Scatter, show, Histogram
from bokeh.layouts import row
output_notebook()

# with open('dataframe.pkl', 'rb') as f:
#     df = pkl.load(f)
    
models = ['regr', 'regr_ey', 'regr_log', 'nb1NN', 'nb2NN', 'nb3NN']

Loading BokehJS ...

In [4]:
# uploading data from files
path = '../messy_data/'
with open(path+'df.pkl', 'rb') as f:
    df_regr = pkl.load(f)
with open(path+'dataframeALL_regr_ey.pkl', 'rb') as f:
    df_ey = pkl.load(f)
# with open(path+'dataframeALL_regr_log.pkl', 'rb') as f:
#     df_log = pkl.load(f)

In [8]:
# NEW DATA
path2 = '../data_final/'
with open(path2+'avg_data.pkl', 'rb') as f:
	avg_data = pkl.load(f)

with open(path2+'ind_models.pkl', 'rb') as f:
    ind_models = pkl.load(f)

In [2]:
# uploading data from folder pkl_data
path = '../pkl_data/'
    
zero = linear_model.LinearRegression()
zero.intercept_, zero.coef_ = 0, np.array([1])

In [3]:
# df_err_*_diff dataframe with differences with and without a model *
with open(path+'df_err_regr.pkl', 'rb') as f:
    df_err_regr = pkl.load(f)
with open(path+'df_err_regr_ey.pkl', 'rb') as f:
    df_err_regr_ey = pkl.load(f)
# with open(path+'df_err_regr_log.pkl', 'rb') as f:
#     df_err_regr_logx = pkl.load(f)
# with open(path+'df_err_nb1NN.pkl', 'rb') as f:
#     df_err_nb1NN = pkl.load(f)
# with open(path+'df_err_nb2NN.pkl', 'rb') as f:
#     df_err_nb2NN = pkl.load(f)
# with open(path+'df_err_nb3NN.pkl', 'rb') as f:
#     df_err_nb3NN = pkl.load(f)
# with open(path+'df_err_nomodel.pkl', 'rb') as f:
#     df_err_nomodel = pkl.load(f)

In [21]:
def plot_coef(models, title, name):
# function for plotting the linear model's intercepts and coefficients
# but it's not working for k-NN models, because they're not lin models
# models is a DataFrame containing models
    X = [x.intercept_ for x in models.values]
    Y = [y.coef_[0] for y in models.values]
    
    f = figure(title=title)
    f.xaxis.axis_label = 'Intercept'
    f.yaxis.axis_label = 'Coefficient'
    f.scatter(x=X, y=Y, color='navy', size=6, alpha=.2)
    show(f)
    print(name+' intercept standard deviation: ', np.std(X))
    print(name+' coefficient standard deviation: ', np.std(Y))
    
def mse_count(df, model):
# function for counting mean squared error for (d_A^2 + d_B^2)/2
    X = [x for x in df[model+' d_A'].values]
    Y = [y for y in df[model+' d_B'].values]
    mse = 0
    for (x,y) in zip(X,Y):
        mse += (x*x + y*y)/2
    return mse/len(X)

def mse_count_dAB(df, model, d='A'):
# function for counting mean squared error for d_A^2
    X = [x for x in df[model+' d_'+d].values]
    mse = 0
    for x in X:
        mse += x*x
    return mse/len(X)

def ind_mse_plot(df, model):
#fun for plotting individual mse errors
    n = 31
    X = [x*x for x in df[model+' d_A'].values]
    Y = [y*y for y in df[model+' d_B'].values]
    f = figure(title="Individual MSE for "+model)
    for i in range(int(len(X)/n)):
        ind_X = X[i*n:i*(n+1)]
        ind_Y = Y[i*n:i*(n+1)]
        mse = 0
        for (x,y) in zip(ind_X, ind_Y):
            mse += (x*x + y*y)/2
        f.scatter(x=mse/n, color='navy', size=6, alpha=.3)
    show(f)

   
##### functions for counting (d_A^2 + d_B^2)/2 and its SD for both regression and zero models

def mean_count(df, model):
# function for counting mean and sd for (d_A^2 + d_B^2)/2
    temp, l = [], []
    for i, (x,y) in enumerate(zip(df[model+' d_A'].values, df[model+' d_B'])):
        temp.append((float(x*x) + float(y*y))/2)
        if (i%31 == 30):
            l.append(np.mean(temp))
            temp = []
    return np.mean(l), np.std(l)

def mean_count_for_zero(ind_models):
# function for counting mean and sd for (d_A^2 + d_B^2)/2 for ZERO MODEL
# d_A^2 = [A_y - A(B^-1(B_y))]^2       for zero model B = A = id
# d_A^2 = [A_y - id(id(B_Y))]^2 = A_y^2 - 2A_yB_y + B_y^2
# (d_A^2 + d_B^2)/2 = A_y^2 - 2A_yB_y + B_y^2

    participants = 241
    N = 31
    temp, l = [], []
    
    for i in range(participants):
        for j in range(i+1,participants):
            for k in range(N):
                A_y, B_y = float(ind_models.loc[i,k]['remain']), float(ind_models.loc[j,k]['remain'])
                temp.append(A_y*A_y - 2*A_y*B_y + B_y*B_y)
            l.append(np.mean(temp))
            temp = []
    return np.mean(l), np.std(l)
                
    

In [22]:
print(mean_count_for_zero(ind_models))
print(mean_count(df_err_regr, 'regr'))
print(mean_count(df_err_regr_ey, 'regr_ey'))

(1866.8594073906731, 1098.989299713528)
(1166.3973610889366, 566.93926173743137)
(3168.0212558924127, 770.75257881736775)


# Plots of intercept and coeficients 

In [12]:
# intercept / coef plots
plot_coef(ind_models['regr'], 'intercept / coefficient '+"regr", "regr")

regr intercept standard deviation:  24.3107320252
regr coefficient standard deviation:  0.204960394309


In [23]:
# intercept / coef plots
plot_coef(ind_models['regr_ey'], 'intercept / coefficient '+"regr_ey", 'regr_ey')

regr_ey intercept standard deviation:  17.1571497595
regr_ey coefficient standard deviation:  2.09078217321e-75


In [24]:
# intercept / coef plots
plot_coef(ind_models['regr_log'], 'intercept / coefficient '+"regr_log", 'regr_log')

regr_log intercept standard deviation:  67.0809899364
regr_log coefficient standard deviation:  15.6521673781


In [18]:
# MSE (d_A^2+d_B^2)/2
for (df, model) in [(df_regr, 'regr'), (df_ey, 'regr_ey')]:
    print('MSE '+model+': ',mse_count(df, model))

MSE regr:  1166.39736109
MSE regr_ey:  3168.02125589


In [ ]:
# MSE d_A^2
for (df, model) in [(df_regr, 'regr'), (df_ey, 'regr_ey'), (df_log, 'regr_log')]:
    print('MSE '+model+': ',mse_count_dAB(df, model, d='A'))

In [ ]:
# MSE d_B^2
for (df, model) in [(df_regr, 'regr'), (df_ey, 'regr_ey'), (df_log, 'regr_log')]:
    print('MSE '+model+': ',mse_count_dAB(df, model, d='B'))

Wyniki uzyskane w lutym (średnie d_A^2)
MSE regr:     1178.05986567
MSE regr_ey:  3200.93215805
MSE regr_log: 473686394475.0
MSE nb1NN:    2111.55143449
MSE nb2NN:    1618.70770597
MSE nb3NN:    1432.190942
MSE zero:     1970.91485889

In [11]:
# MSE dopasowania poszczególnych modeli dla każdego uczestnika (też z walidacją leave-one-out).
# mean(mse(i) for i in participants) = mse(participants)

def model_mse_sd(models):
    #### mse: A^-1(y_A) comparing to avg answers
    temp = []
    temp2, ind_mse2 = [], []
    remains = ind_models['remain'].values
    for i, lin_mod in enumerate(models):
        temp.append(lin_mod.predict(float(remains[i])))
        temp2.append(lin_mod.predict(float(remains[i])))
        if (i%31 == 30):
            ind_mse2.append(mse(temp2, avg_data['mean']))
            temp2 = []
    return mse(temp, list(avg_data['mean'])*int(len(remains)/31)), np.std(temp), np.mean(ind_mse2), np.std(ind_mse2)

print('model mse and sd regr:    ', model_mse_sd(ind_models['inv regr'].values))
print('model mse and sd regr_ey: ', model_mse_sd(ind_models['inv regr_ey'].values))
print('model mse and sd zero:    ', model_mse_sd(np.array([zero]*7471)))

model mse and sd regr:     (506.68580446655898, 43.304732143037839, 506.68580446655898, 297.34583889368315)
model mse and sd regr_ey:  (2490.2612022371291, 3.2396856154448601, 2490.2612022371291, 55.360968182030071)
model mse and sd zero:     (929.55655139784551, 56.941449720228405, 929.55655139784574, 590.4938971421011)


In [ ]:
# TODO liczy się w nieskończoność
ind_mse_plot(df_regr, 'regr')

In [ ]:
# models plots TESTS
""""def plot_ind_model(model, plot_title):
    f = figure(title=plot_title)
    X = avg_data['stimulus']
    Y = 
#     model[]
    show(f)
 """"   

In [19]:
def compare_ind_zero_model(inv_ind_models, remains):
#### comparing communication error with and without individual model####
# ind_model is a DataFrame containing one model / it's a column ['model_name'] from ind_models
# column['remain'] from ind_models

    # participants = len(data_all)/31
    # N = len(data_all[0])
    participants = 243
    N = 31

    zero_model_diff = []
    ind_model_diff = []

    for i in range(participants):
        for j in range(i+1,participants):
            for k in range(N):
                A, B = inv_ind_models.loc[i,k], inv_ind_models.loc[j,k]
                A_y, B_y = float(remains.loc[i,k]), float(remains.loc[j,k])
                # diff between raw answers
                zero_model_diff.append(abs(A_y - B_y))
                #  diff between answers with individual models
                ind_model_diff.append(abs(float(A.predict(A_y) - B.predict(B_y))))
        print(i)
    # dif = [zero-ind for zero, ind in zip(zero_model_diff,ind_model_diff)]
    # ind_zero_mse = mse(zero_model_diff, ind_model_diff)
    # return np.mean(dif), ind_zero_mse
    return zero_model_diff, ind_model_diff

def compare_ind_zero_model2(ind_models, inv_ind_models, remains):
#### comparing communication error with and without individual model####
# ind_model is a DataFrame containing one model / it's a column ['model_name'] from ind_models
# column['remain'] from ind_models

# here it is B(A^-1(A_y)) - B_y

    # participants = len(data_all)/31
    # N = len(data_all[0])
    participants = 243
    N = 31

    zero_model_diff = []
    ind_model_diff = []

    for i in range(participants):
        for j in range(i+1,participants):
            for k in range(N):
                A, B = inv_ind_models.loc[i,k], ind_models.loc[j,k]
                A_y, B_y = float(remains.loc[i,k]), float(remains.loc[j,k])
                # diff between raw answers
                zero_model_diff.append(abs(A_y - B_y))
                #  diff between answers with individual models
                ind_model_diff.append(abs(float(B.predict(float(A.predict(A_y)) - B_y))))
                
    # dif = [zero-ind for zero, ind in zip(zero_model_diff,ind_model_diff)]
    # ind_zero_mse = mse(zero_model_diff, ind_model_diff)
    # return np.mean(dif), ind_zero_mse
    return zero_model_diff, ind_model_diff

In [ ]:
f = figure(title=title)
f.xaxis.axis_label = 'Intercept'
f.yaxis.axis_label = 'Coefficient'
f.scatter(x=zero_model_diff, y=Y, color='navy', size=6, alpha=.2)
show(f)
print('intercept standard deviation: ', np.std(X))
print('coefficient standard deviation: ', np.std(Y))

# REGR histograms

In [20]:
inv_regr = ind_models['inv regr']
remains = ind_models['remain']
(zero_model_diff, ind_model_diff) = compare_ind_zero_model(inv_regr, remains)

#  B(A(A_y))
regr = ind_models['regr']
ind_model_diff2 = compare_ind_zero_model2(regr, inv_regr, remains)[1]

zero_ind_diff = [zero-ind for (zero, ind) in zip(zero_model_diff, ind_model_diff)]
zero_ind_diff2 = [zero-ind for (zero, ind) in zip(zero_model_diff, ind_model_diff2)]

zero_regr_ey_diff = [zero-regr_ey for (zero, regr_ey) in zip(zero_model_diff, ind_regr_ey_diff)]

KeyError: 'the label [241] is not in the [index]'

In [ ]:
#### histograms presenting the communication differences between to agents
# REGR

def histogram(data, bins, title):
    
    p = Histogram(data, title=title, bins=bins, xgrid=True)
    p.xaxis.axis_label = 'difference'
    p.yaxis.axis_label = 'occurring'
    p.y_range.end = 60000
    p.x_range.start = 0
    p.x_range.end = 200
    return p

data1 = ind_model_diff
bins1 = int(max(data1)/2)
p1 = histogram(data1, bins1, "Individual regr difference")

data2 = ind_model_diff2
bins2 = int(max(data2)/2)
p2 = histogram(data2, bins2, "Individual regr2 difference")

data3 = zero_model_diff
bins3 = int(max(data3)/2)
p3 = histogram(data3, bins3, "Zero model difference")


show(row(p3, p1, p2))

In [ ]:
data4 = zero_ind_diff
bins4 = int((max(data4) - min(data4))/2)
p4 = Histogram(data4, title='Zero model difference - Ind regr difference', bins=bins4, xgrid=True)
p4.xaxis.axis_label = 'difference'
p4.yaxis.axis_label = 'occurring'
p4.y_range.end = 60000
p4.x_range.start = -150
p4.x_range.end = 150

data5 = zero_ind_diff2
bins5 = int((max(data5) - min(data5))/2)
p5 = Histogram(data5, title='Zero model difference - Ind regr difference2', bins=bins5, xgrid=True)
p5.xaxis.axis_label = 'difference'
p5.yaxis.axis_label = 'occurring'
p5.y_range.end = 60000
p5.x_range.start = -150
p5.x_range.end = 150

show(row(p4,p5))


In [ ]:
# difference between ind_model_diff(A(A_y)-B(B_Y)) and ind_model_diff2(B(A(A_Y))-B_y)
data6 = [ind1 - ind2 for (ind1, ind2) in zip(ind_model_diff, ind_model_diff2)]
bins6 = int((max(data6) - min(data6))/2)

p6 = Histogram(between, title='Ind regr difference - Ind regr difference 2', bins=bins6, xgrid=True)
p6.xaxis.axis_label = 'difference'
p6.yaxis.axis_label = 'occurring'
p6.y_range.end = 80000
p6.x_range.start = -150
p6.x_range.end = 150

show(p6)

# REGR_EY histograms

In [ ]:
inv_regr_ey = ind_models['inv regr_ey']
ind_regr_ey_diff = compare_ind_zero_model(inv_regr_ey, remains)[1]

In [ ]:
#### histograms presenting the communication differences between to agents
# REGR_EY

# data7 = ind_regr_ey_diff
# bins7 = int(max(data7)/2)
# p7 = histogram(data7, bins7, "Individual regr_ey difference")
# show(p7)
# ''''''''''
p7 = Histogram(ind_regr_ey_diff, title='Individual regr_ey difference', bins=200, xgrid=True)
p7.xaxis.axis_label = 'difference'
p7.yaxis.axis_label = 'occurring'
p7.y_range.end = 200000
p7.x_range.start = 0
p7.x_range.end = 50

show(p7)
# '''''''''


In [ ]:
# zero model - regr_ey

data8 = zero_regr_ey_diff
bins8 = int((max(data8) - min(data8))/2)
p8 = Histogram(data8, title='Zero model difference - Ind regr_ey difference', bins=bins4, xgrid=True)
p8.xaxis.axis_label = 'difference'
p8.yaxis.axis_label = 'occurring'
p8.y_range.end = 60000
p8.x_range.start = -150
p8.x_range.end = 150

show(p8)

In [ ]:
#### mean difference for histograms above

print('mean for zero model diff:       ', np.mean(zero_model_diff))
print('mean for ind regr diff:         ', np.mean(ind_model_diff))
print('mean for (zero - regr) diff:    ', np.mean(zero_ind_diff))
print('mean for ind regr2 diff:        ', np.mean(ind_model_diff2))
print('mean for (zero - regr2) diff    ', np.mean(zero_ind_diff2))
print('mean for ind regr_ey diff:      ', np.mean(ind_regr_ey_diff))
#### standard deviation for histograms above
print('SD for zero model diff:         ', np.std(zero_model_diff))
print('SD for ind regr diff:           ', np.std(ind_model_diff))
print('SD for (zero - regr) diff:      ', np.std(zero_ind_diff))
print('SD for ind regr2 diff:          ', np.std(ind_model_diff2))
print('SD for (zero - regr2) diff      ', np.std(zero_ind_diff2))
print('SD for ind regr_ey diff:        ', np.std(ind_regr_ey_diff))

In [ ]:
# TESTS
import scipy.special

from bokeh.layouts import gridplot

p1 = figure(title="Normal Distribution (μ=0, σ=0.5)",tools="save",
            background_fill_color="#E8DDCB")

xx = [1,1,2,3,0,0,0,0,-1,-1,-3,1, 1.1, 3]

mu, sigma = 0, 0.5

measured = np.random.normal(mu, sigma, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(-2, 2, 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2
hist = np.histogram(ind_model_diff, density=True, bins=50)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
#p1.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="PDF")
#p1.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p1.legend.location = "top_left"
p1.xaxis.axis_label = 'x'
p1.yaxis.axis_label = 'Pr(x)'


In [ ]:
bins = int(max(zero_regr_ey_diff)/2)
p3 = Histogram(zero_regr_ey_diff, title='Zero model difference - Ind regr_ey difference', bins=bins, xgrid=True)
p3.xaxis.axis_label = 'difference'
p3.yaxis.axis_label = 'occurring'
p3.y_range.end = 60000
p3.x_range.start = -10
p3.x_range.end = 200

show(p3)